# Tablica wyników

https://klejbenchmark.com/leaderboard/

https://github.com/allegro/klejbenchmark-baselines

In [ ]:
!git clone https://github.com/allegro/klejbenchmark-baselines

Cloning into 'klejbenchmark-baselines'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 1), reused 19 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [ ]:
!pip install klejbenchmark-baselines/

Processing ./klejbenchmark-baselines
     |████████████████████████████████| 17.4MB 209kB/s 
     |████████████████████████████████| 10.1MB 56.3MB/s 
     |████████████████████████████████| 5.4MB 44.3MB/s 
     |████████████████████████████████| 24.8MB 1.4MB/s 
     |████████████████████████████████| 573kB 48.2MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 753.4MB 23kB/s 
     |████████████████████████████████| 204kB 59.8MB/s 
     |████████████████████████████████| 133kB 55.0MB/s 
     |████████████████████████████████| 890kB 54.4MB/s 
     |████████████████████████████████| 3.7MB 44.0MB/s 
     |████████████████████████████████| 1.2MB 48.8MB/s 
     |████████████████████████████████| 829kB 47.1MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 7.2MB 23.1MB/s 
  Created wheel for klejbenchmark-baselines: filename=klejbenchmark_baselines-0.1.0-cp36-none-any.whl size=15196 sha256=6a336

In [ ]:
!ls klejbenchmark-baselines

klejbenchmark_baselines  README.md	   scripts
LICENSE			 requirements.txt  setup.py


In [ ]:
!ls klejbenchmark-baselines/scripts

download_klej.sh  run_training.sh


In [ ]:
!cat klejbenchmark-baselines/scripts/download_klej.sh

#!/bin/bash
set -euo pipefail
IFS=$'\n\t'

# User provided arguments
DATA_PATH="klej_data"

# Create output dir
mkdir -p "${DATA_PATH}"

# Download KLEJ datasets
task_names=("nkjp-ner" "cdsc-e" "cdsc-r" "cbd" "polemo2.0-in" "polemo2.0-out" "dyk" "psc" "ar")
for task_name in "${task_names[@]}"; do
    echo "Downloading: ${task_name}"
    curl -o "${DATA_PATH}/klej_${task_name}.zip" "https://klejbenchmark.com/static/data/klej_${task_name}.zip"
    unzip "${DATA_PATH}/klej_${task_name}.zip" -d "${DATA_PATH}/klej_${task_name}/"
    rm "${DATA_PATH}/klej_${task_name}.zip"
done
echo "Done!"


In [ ]:
!cat klejbenchmark-baselines/scripts/run_training.sh

#!/bin/bash
set -euo pipefail
IFS=$'\n\t'

# User provided arguments
RUN_ID_PREFIX="klej_herbert"
DATA_PATH="klej_data"
OUTPUT_PATH="output"
TOKENIZER_NAME_OR_PATH="allegro/herbert-klej-cased-tokenizer-v1"  # local path or the name of the transformers tokenizer
MODEL_NAME_OR_PATH="allegro/herbert-klej-cased-v1"  # local path or the name of the transformers model

task_names=("nkjp-ner" "cdsc-e" "cdsc-r" "cbd" "polemo2.0-in" "polemo2.0-out" "dyk" "psc" "ar")
run_date="$(date +%Y%m%d_%H%M%S)"
run_id="${RUN_ID_PREFIX}_${run_date}"

# Create output dir
mkdir -p "${OUTPUT_PATH}/submissions/${run_id}/"

# Train
for task_name in "${task_names[@]}"; do
    task_path="${DATA_PATH}/klej_${task_name}"

    # PSC task requires longer sequences
    if [[ "${task_name}" == "psc" ]]; then
        max_len=510
        batch_size=8
        gradient_accumulation_steps=4
    else
        max_len=256
        batch_size=16
        gradient_accumulation_steps=2
    fi

    # Run task training
    python klej

In [ ]:
!bash klejbenchmark-baselines/scripts/download_klej.sh

Downloading: nkjp-ner
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  802k  100  802k    0     0  1103k      0 --:--:-- --:--:-- --:--:-- 1102k
Archive:  klej_data/klej_nkjp-ner.zip
  inflating: klej_data/klej_nkjp-ner/dev.tsv  
  inflating: klej_data/klej_nkjp-ner/test_features.tsv  
  inflating: klej_data/klej_nkjp-ner/train.tsv  
Downloading: cdsc-e
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  367k  100  367k    0     0   586k      0 --:--:-- --:--:-- --:--:--  586k
Archive:  klej_data/klej_cdsc-e.zip
  inflating: klej_data/klej_cdsc-e/dev.tsv  
  inflating: klej_data/klej_cdsc-e/test_features.tsv  
  inflating: klej_data/klej_cdsc-e/train.tsv  
Downloading: cdsc-r
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
          

In [ ]:
!bash klejbenchmark-baselines/scripts/run_training.sh

python3: can't open file 'klejbenchmark_baselines/main.py': [Errno 2] No such file or directory


In [ ]:
!mkdir klej_data/submissions

In [ ]:
%%bash

python klejbenchmark-baselines/klejbenchmark_baselines/main.py \
      --run-id "123" \
      --task-name "ar" \
      --task-path "klej_data/klej_ar/" \
      --predict-path "klej_data/submissions/test_pred_ar.tsv" \
      --logger-path "klej_data/klej_ar/tb/" \
      --checkpoint-path "klej_data/klej_ar/checkpoints/" \
      --tokenizer-name-or-path "allegro/herbert-klej-cased-tokenizer-v1" \
      --model-name-or-path "allegro/herbert-klej-cased-v1" \
      --max-seq-length "256" \
      --batch-size "16" \
      --gradient-accumulation-steps "2" \
      --num-gpu 1

Epoch 4: 100%|██████████| 662/662 [08:11<00:00,  1.35it/s, loss=0.106, lr=0, train_loss=0.65424454, v_num=ar, val_loss=0.632, val_wmae=0.837]

Predicting: 100%|██████████| 63/63 [00:18<00:00,  3.42it/s]


INFO:transformers.file_utils:PyTorch version 1.4.0 available.
2021-01-16 08:18:48.214826: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
INFO:transformers.file_utils:TensorFlow version 2.4.0 available.
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/allegro/herbert-klej-cased-tokenizer-v1/config.json from cache at /root/.cache/torch/transformers/2a06d252aa1beec2b689682613a6873c248fafc3c8a227b2d5b8d41bbc743d28.6f4a30a569f6e45abd8a13cd2316e49bbeaebea8275e12fd44d4d3db8a0cdbcb
INFO:transformers.configuration_utils:Model config XLMConfig {
  "_num_labels": 2,
  "architectures": null,
  "asm": false,
  "attention_dropout": 0.1,
  "bad_words_ids": null,
  "bos_index": 0,
  "bos_token_id": 0,
  "causal": false,
  "decoder_start_token_id": null,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "emb_dim": 2048,
  "embed_init_std": 0.02209

In [ ]:
!head klej_data/submissions/test_pred_ar.tsv

target
3.727707
5.115761
3.4707904
5.2536697
5.36901
2.9499714
1.049658
5.1176033
5.1549273
